In [1]:
import numpy as np
import pickle
import glob
import matplotlib.pyplot as plt
import pandas as pd
from scipy import *
import os
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Conv1D,MaxPooling1D,Flatten,BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn import *
from sklearn.metrics import *
sns.set()
import warnings
warnings.filterwarnings("ignore")
length = 277

2024-03-26 19:27:34.004833: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-26 19:27:34.027543: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-26 19:27:34.027561: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-26 19:27:34.028673: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-26 19:27:34.033012: I tensorflow/core/platform/cpu_feature_guar

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17623689598673955715
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5803868160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8950066975517240723
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


2024-03-26 19:27:34.912667: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 19:27:34.932270: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 19:27:34.932310: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 19:27:35.024934: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 19:27:35.024979: I external/local_xla/xla/stream_executor

In [3]:
train_values = pd.read_csv('./data/train_test/train_patients.csv').values
test_values = pd.read_csv('./data/train_test/test_patients.csv').values
    
print(train_values.shape)
print(test_values.shape)

# Separate the training and testing data, and one-hot encode Y:
X_train = train_values[:,:-2]
X_test = test_values[:,:-2]
y_train = train_values[:,-2]
y_test = test_values[:,-2]
X_test1 = X_test.reshape(-1, X_train.shape[1], 1)
y_test1=to_categorical(y_test)


X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)
X_train1 = X_train.reshape(-1, X_train.shape[1], 1)
y_train1=to_categorical(y_train)

X_val1 = X_val.reshape(-1, X_train.shape[1], 1)
y_val1=to_categorical(y_val)


(38751, 277)
(72794, 277)
X_train shape: (31000, 275)
X_val shape: (7751, 275)
y_train shape: (31000,)
y_val shape: (7751,)


In [4]:
# Initialize the model
cnnmodel = Sequential()

# Ensure the input_shape parameter matches the shape of X_train1
cnnmodel.add(Conv1D(396, 5, strides=1,padding='same', activation='relu', input_shape=(X_train1.shape[1],X_train1.shape[2])))
cnnmodel.add(BatchNormalization())

cnnmodel.add(Conv1D(192, 7, strides=1,padding='same', activation='relu'))

cnnmodel.add(BatchNormalization())
cnnmodel.add(Conv1D(88, 9, strides=1,padding='same', activation='relu'))
cnnmodel.add(BatchNormalization())
cnnmodel.add(Conv1D(34, 11, strides=1,padding='same', activation='relu', name = 'last_conv_layer'))
cnnmodel.add(MaxPooling1D(pool_size=2, strides=2))

# Flatten the tensor output from the convolutional part of the CNN
cnnmodel.add(Flatten())

# Add fully connected layers
cnnmodel.add(Dense(30, activation='relu'))
cnnmodel.add(Dense(20, activation='relu'))
cnnmodel.add(Dense(7, activation='softmax'))  

cnnmodel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 275, 396)          2376      
                                                                 
 batch_normalization (Batch  (None, 275, 396)          1584      
 Normalization)                                                  
                                                                 
 conv1d_1 (Conv1D)           (None, 275, 192)          532416    
                                                                 
 batch_normalization_1 (Bat  (None, 275, 192)          768       
 chNormalization)                                                
                                                                 
 conv1d_2 (Conv1D)           (None, 275, 88)           152152    
                                                                 
 batch_normalization_2 (Bat  (None, 275, 88)           3

2024-03-26 19:27:36.836154: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 19:27:36.836217: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 19:27:36.836233: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 19:27:36.836338: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 19:27:36.836357: I external/local_xla/xla/stream_executor

In [7]:
cnnmodel.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode='min',min_delta=0.001, restore_best_weights=True)
history = cnnmodel.fit(X_train1, y_train1, validation_data=(X_val1, y_val1), epochs=50,callbacks=[early_stopping], verbose=1)


Epoch 1/50


2024-03-26 19:27:43.960600: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-26 19:27:44.102112: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-26 19:27:44.656626: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-26 19:27:45.257180: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f8e8d3421d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-26 19:27:45.257223: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-03-26 19:27:45.261756: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1711481265.310543  102322 devic

969/969 [==============================] - 17s 13ms/step - loss: 0.2431 - accuracy: 0.9269 - val_loss: 0.1314 - val_accuracy: 0.9587
Epoch 2/50
969/969 [==============================] - 12s 12ms/step - loss: 0.1085 - accuracy: 0.9669 - val_loss: 0.1082 - val_accuracy: 0.9663
Epoch 3/50
969/969 [==============================] - 12s 12ms/step - loss: 0.0850 - accuracy: 0.9735 - val_loss: 0.1011 - val_accuracy: 0.9715
Epoch 4/50
969/969 [==============================] - 12s 12ms/step - loss: 0.0635 - accuracy: 0.9795 - val_loss: 0.0695 - val_accuracy: 0.9799
Epoch 5/50
969/969 [==============================] - 12s 12ms/step - loss: 0.0513 - accuracy: 0.9831 - val_loss: 0.0848 - val_accuracy: 0.9781
Epoch 6/50
969/969 [==============================] - 12s 12ms/step - loss: 0.0460 - accuracy: 0.9854 - val_loss: 0.0671 - val_accuracy: 0.9799
Epoch 7/50
969/969 [==============================] - 12s 12ms/step - loss: 0.0372 - accuracy: 0.9879 - val_loss: 0.0857 - val_accuracy: 0.9757
Epo

In [8]:
class_counts = np.bincount(y_test.astype(int))
print(class_counts)


[    0 58951  2493  6725  2923  1277   425]


In [9]:
from sklearn.metrics import confusion_matrix, f1_score

# Predict the test data
y_pred = cnnmodel.predict(X_test1)

# Convert the predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Create the confusion matrix
cm = confusion_matrix(y_test, y_pred_labels)
print("Confusion Matrix:")
print(cm)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred_labels, average='weighted')
print("F1 Score:", f1)




2275/2275 [==============================] - 6s 3ms/step
Confusion Matrix:
[[58676     0    19    71   159    26]
 [    0  2492     1     0     0     0]
 [   39     0  6663     9    12     2]
 [   44     0     9  2854     2    14]
 [   64     0    10     1  1202     0]
 [    0     0     1     1     0   423]]
F1 Score: 0.9934256327252254


In [11]:
cnnmodel.save('./models/cnnmodel_grad.h5')
